In [2]:
#.1

from datetime import datetime
from datetime import datetime


# Set the chromedriver path
webdriver_path = "chromedriver.exe"

# Open the Chrome window
browser = webdriver.Chrome(executable_path=webdriver_path)

browser.get("https://www.expedia.com/Flights-Search?=one-key-onboarding-dialog&d1=2023-05-22&flight-type=on&fromDate=5%2F22%2F2023&leg1=from%3AWarsaw%20%28WAW%20-%20Frederic%20Chopin%29%2Cto%3ASingapore%20%28SIN%20-%20Changi%29%2Cdeparture%3A5%2F22%2F2023TANYT&mode=search&options=cabinclass%3Aeconomy&passengers=children%3A0%2Cadults%3A1%2Cseniors%3A0%2Cinfantinlap%3AY&trip=oneway")


# Set Round Trip type path
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"

#2 define a few functions using Selenium tools in order to find various features on the webpage. The function’s names imply about their role.

import time
from selenium.webdriver.common.keys import Keys

def ticket_chooser(ticket):
    try:
        ticket_type = browser.find_element_by_xpath(ticket)
        ticket_type.click()
    except Exception as e:
        pass

def more_details(details):
    try:
        details_type = browser.find_element_by_xpath(details)
        details_type.click()
    except Exception as e:
        pass

def dep_country_chooser(dep_country):
    fly_from = browser.find_element_by_xpath("//input[@id='flight-origin-hp-flight']")
    time.sleep(3)
    fly_from.clear()
    time.sleep(3)
    fly_from.send_keys('  ' + dep_country)
    time.sleep(3)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(3)
    first_item.click()

def arrival_country_chooser(arrival_country):
    fly_to = browser.find_element_by_xpath("//input[@id='flight-destination-hp-flight']")
    time.sleep(3)
    fly_to.clear()
    time.sleep(3)
    fly_to.send_keys('  ' + arrival_country)
    time.sleep(3)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(3)
    first_item.click()

def dep_date_chooser(month, day, year):
    dep_date_button = browser.find_element_by_xpath("//input[@id='flight-departing-hp-flight']")
    dep_date_button.clear()
    dep_date_button.send_keys(str(month) + "/" + str(day) + "/" + str(year))

def return_date_chooser(month, day, year):
    return_date_button = browser.find_element_by_xpath("//input[@id='flight-returning-hp-flight']")
    for i in range(11):
        return_date_button.send_keys(Keys.BACKSPACE)
    return_date_button.send_keys(str(month) + "/" + str(day) + "/" + str(year))


#3. 
def ProcessJourney(journeyDetails, i):
    ChooseFlight(journeyDetails, i)
    DataProcessing()
    AmendTimeAndDate()
    SaveDfToCsv(journeyDetails, i)
    
    # Customization: Perform additional operations for a specific case
    if journeyDetails['departure_city'] == 'Warsaw' and journeyDetails['arrival_city'] == 'Singapore':
        # Perform some special operations or calculations
        # Example: Calculate the total travel time for the journey
        total_travel_time = journeyDetails['arrival_time'] - journeyDetails['departure_time']
        journeyDetails['total_travel_time'] = total_travel_time
    
    # Continue with other operations or customizations if needed


#4. 

def DataProcessing():
    global df
    df = df[0:0]
    number_element_to_ignore = 0   
    
    # Departure times
    dep_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
    dep_times_list = [value.text for value in dep_times]
    
    # Arrival times
    arr_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
    arr_times_list = [value.text for value in arr_times]
    
    # Airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]
        
    # Durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]
    
    # Stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]
    
    # Layovers
    layovers = browser.find_elements_by_xpath("//span[@data-test-id='layover-airport-stops']")
    layovers_list = [value.text for value in layovers]
    
    # Prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list = [value.text for value in prices]
    
    # Last flight to scrape according to the price differences (here we choose multiply by 2)  
    last_flight_index = CheckLastFlightIndexByPrice(price_list, 2)
    
    # Generate flights to ignore according to the last_flight_index
    bad_indexes = GenerateBadIndex(dep_times_list, last_flight_index)

    # Delete the non-relevant flights
    for i in range(len(bad_indexes)):
        dep_times_list.pop(int(bad_indexes[i]))
        arr_times_list.pop(int(bad_indexes[i]))
    
    # Insert data to our DataFrame
    for i in range(last_flight_index):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'price'] = price_list[i]
        except Exception as e:
            pass
        try:
            # Adding flight details data
            number_element_to_ignore = AddFlightDetailsData(i, stops_list, bad_indexes, number_element_to_ignore)
        except Exception as e:
            pass
#Replace the AddingFlightDetails function call with AddFlightDetailsData or the appropriate function or code that adds flight details data.


#5.

import os

def SaveDfToCsv(journeyDetails, i):
    [folderPath, conc_date] = GetPathForExcelsOutPut(journeyDetails, i)
    documents_folder = os.path.expanduser(".")
    output_folder = os.path.join(documents_folder, folderPath)
    os.makedirs(output_folder, exist_ok=True)
    file_name = os.path.join(output_folder, f"{conc_date}_{dt.datetime.today().strftime('%y%m%d-%H%M%S')}.csv")
    df['departure_date'] = GetDepartDateUsingFolderName(conc_date)  # Update with appropriate function
    df['arrival_date'] = GetReturnDateUsingFolderName(conc_date)  # Update with appropriate function
    df.to_csv(file_name, index=False)


NameError: name 'webdriver' is not defined

In [ ]:
# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import selenium.common.exceptions as selexcept

# Pandas imports
import pandas as pd
from datetime import datetime
import os.path
import re
import sys
import glob

# Set the chromedriver path
webdriver_path = "chromedriver.exe"

# Open the Chrome window
browser = webdriver.Chrome(executable_path=webdriver_path)

browser.get("https://www.expedia.com/Flights-Search?=one-key-onboarding-dialog&d1=2023-05-22&flight-type=on&fromDate=5%2F22%2F2023&leg1=from%3AWarsaw%20%28WAW%20-%20Frederic%20Chopin%29%2Cto%3ASingapore%20%28SIN%20-%20Changi%29%2Cdeparture%3A5%2F22%2F2023TANYT&mode=search&options=cabinclass%3Aeconomy&passengers=children%3A0%2Cadults%3A1%2Cseniors%3A0%2Cinfantinlap%3AY&trip=oneway")


# Set Round Trip type path
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"

# Define a few functions using Selenium tools to find various features on the webpage

import time
from selenium.webdriver.common.keys import Keys

def ticket_chooser(ticket):
    try:
        ticket_type = browser.find_element(By.XPATH, ticket)
        ticket_type.click()
    except Exception as e:
        pass

def more_details(details):
    try:
        details_type = browser.find_element(By.XPATH, details)
        details_type.click()
    except Exception as e:
        pass

def dep_country_chooser(dep_country):
    fly_from = browser.find_element(By.XPATH, "//input[@id='flight-origin-hp-flight']")
    time.sleep(3)
    fly_from.clear()
    time.sleep(3)
    fly_from.send_keys('  ' + dep_country)
    time.sleep(3)
    first_item = browser.find_element(By.XPATH, "//a[@id='aria-option-0']")
    time.sleep(3)
    first_item.click()

def arrival_country_chooser(arrival_country):
    fly_to = browser.find_element(By.XPATH, "//input[@id='flight-destination-hp-flight']")
    time.sleep(3)
    fly_to.clear()
    time.sleep(3)
    fly_to.send_keys('  ' + arrival_country)
    time.sleep(3)
    first_item = browser.find_element(By.XPATH, "//a[@id='aria-option-0']")
    time.sleep(3)
    first_item.click()

def dep_date_chooser(month, day, year):
    dep_date_button = browser.find_element(By.XPATH, "//input[@id='flight-departing-hp-flight']")
    dep_date_button.clear()
    dep_date_button.send_keys(str(month) + "/" + str(day) + "/" + str(year))

def return_date_chooser(month, day, year):
    return_date_button = browser.find_element(By.XPATH, "//input[@id='flight-returning-hp-flight']")
    for i in range(11):
        return_date_button.send_keys(Keys.BACKSPACE)
    return_date_button.send_keys(str(month) + "/" + str(day) + "/" + str(year))


# Define the ProcessJourney function

def ProcessJourney(journeyDetails, i):
    ChooseFlight(journeyDetails,i)
    DataProcessing()
    AmendTimeAndDate()
    SaveDfToCsv(journeyDetails, i)
    
    # Customization: Perform additional operations for a specific case
    if journeyDetails['departure_city'] == 'Warsaw' and journeyDetails['arrival_city'] == 'Singapore':
        # Perform some special operations or calculations
        # Example: Calculate the total travel time for the journey
        total_travel_time = journeyDetails['arrival_time'] - journeyDetails['departure_time']
        journeyDetails['total_travel_time'] = total_travel_time
    
    # Continue with other operations or customizations if needed


# Data Processing function

def DataProcessing():
    global df
    df = pd.DataFrame(columns=['departure_time', 'arrival_time', 'airline', 'duration', 'stops', 'layovers', 'price'])
    
    number_element_to_ignore = 0   
    
    # Departure times
    dep_times = browser.find_elements(By.XPATH, "//span[@data-test-id='departure-time']")
    dep_times_list = [value.text for value in dep_times]
    
    # Arrival times
    arr_times = browser.find_elements(By.XPATH, "//span[@data-test-id='arrival-time']")
    arr_times_list = [value.text for value in arr_times]
    
    # Airline name
    airlines = browser.find_elements(By.XPATH, "//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]
        
    # Durations
    durations = browser.find_elements(By.XPATH, "//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]
    
    # Stops
    stops = browser.find_elements(By.XPATH, "//span[@class='number-stops']")
    stops_list = [value.text for value in stops]
    
    # Layovers
    layovers = browser.find_elements(By.XPATH, "//span[@data-test-id='layover-airport-stops']")
    layovers_list = [value.text for value in layovers]
    
    # Prices
    prices = browser.find_elements(By.XPATH, "//span[@data-test-id='listing-price-dollars']")
    price_list = [value.text for value in prices]
    
    # Last flight to scrape according to the price differences (here we choose multiply by 2)  
    last_flight_index = CheckLastFlightIndexByPrice(price_list, 2)
    
    # Generate flights to ignore according to the last_flight_index
    bad_indexes = GenerateBadIndex(dep_times_list, last_flight_index)

    # Delete the non-relevant flights
    for i in range(len(bad_indexes)):
        dep_times_list.pop(int(bad_indexes[i]))
        arr_times_list.pop(int(bad_indexes[i]))
    
    # Insert data into our DataFrame
    for i in range(last_flight_index):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'price'] = price_list[i]
        except Exception as e:
            pass
        try:
            # Adding flight details data
            number_element_to_ignore = Add
####
# 2.0
# Define the AddFlightDetailsData function

def AddFlightDetailsData(i, stops_list, bad_indexes, number_element_to_ignore):
    # Code to add flight details data
    for i in range(len(stops_list)):
        if i not in bad_indexes:
            try:
                stops = int(stops_list[i])
                if stops > number_element_to_ignore:
                    # Add your code to process and add flight details data here
                    # Example: Extract layover details, flight numbers, etc.
                    # And store the information in the respective columns of the DataFrame
                    pass
            except Exception as e:
                pass



# Define the ChooseFlight function

def ChooseFlight(journeyDetails, i):
    # Code to choose the flight
    ticket_chooser(return_ticket)
    more_details(details)
    dep_country_chooser(journeyDetails['departure_city'])
    arrival_country_chooser(journeyDetails['arrival_city'])
    dep_date_chooser(journeyDetails['departure_date'].month, journeyDetails['departure_date'].day, journeyDetails['departure_date'].year)
    return_date_chooser(journeyDetails['return_date'].month, journeyDetails['return_date'].day, journeyDetails['return_date'].year)
    # Add any additional code to customize flight selection if needed

# Define the remaining functions as mentioned in the previous responses

# 2.4

# Define the remaining functions

def AmendTimeAndDate():
    # Code to amend time and date format
    pass

def SaveDfToCsv(journeyDetails, i):
    # Code to save DataFrame to CSV file
    pass

def CheckLastFlightIndexByPrice(price_list, multiplier):
    # Code to check the index of the last flight based on price differences
    return 0

def GenerateBadIndex(dep_times_list, last_flight_index):
    # Code to generate bad indexes based on specific conditions
    return []

def AddFlightDetailsData(i, stops_list, bad_indexes, number_element_to_ignore):
    # Code to add flight details data
    for i in range(len(stops_list)):
        if i not in bad_indexes:
            try:
                stops = int(stops_list[i])
                if stops > number_element_to_ignore:
                    # Add your code to process and add flight details data here
                    # Example: Extract layover details, flight numbers, etc.
                    # And store the information in the respective columns of the DataFrame
                    pass
            except Exception as e:
                pass

# Define the ChooseFlight function

def ChooseFlight(journeyDetails, i):
    # Code to choose the flight
    ticket_chooser(return_ticket)
    more_details(details)
    dep_country_chooser(journeyDetails['departure_city'])
    arrival_country_chooser(journeyDetails['arrival_city'])
    dep_date_chooser(journeyDetails['departure_date'].month, journeyDetails['departure_date'].day, journeyDetails['departure_date'].year)
    return_date_chooser(journeyDetails['return_date'].month, journeyDetails['return_date'].day, journeyDetails['return_date'].year)
    # Add any additional code to customize flight selection if needed

# Call the ProcessJourney function with appropriate journey details

journeyDetails = {
    'departure_city': 'Warsaw',
    'arrival_city': 'Singapore',
    'departure_date': datetime(2023, 5, 22),
    'return_date': datetime(2023, 5, 29)
}

ProcessJourney(journeyDetails, 0)


# 3.0
# Define the AddFlightDetailsData function

def AddFlightDetailsData(i, stops_list, bad_indexes, number_element_to_ignore):
    # Code to add flight details data
    for i in range(len(stops_list)):
        if i not in bad_indexes:
            try:
                stops = int(stops_list[i])
                if stops > number_element_to_ignore:
                    # Add your code to process and add flight details data here
                    # Example: Extract layover details, flight numbers, etc.
                    # And store the information in the respective columns of the DataFrame
                    layovers = layovers_list[i].split(' ')
                    layovers = [layover.strip() for layover in layovers if layover.strip() != '-']
                    df.loc[i, 'layovers'] = ', '.join(layovers)
            except Exception as e:
                pass

# Define the ChooseFlight function

def ChooseFlight(journeyDetails, i):
    # Code to choose the flight
    ticket_chooser(return_ticket)
    more_details(details)
    dep_country_chooser(journeyDetails['departure_city'])
    arrival_country_chooser(journeyDetails['arrival_city'])
    dep_date_chooser(journeyDetails['departure_date'].month, journeyDetails['departure_date'].day, journeyDetails['departure_date'].year)
    return_date_chooser(journeyDetails['return_date'].month, journeyDetails['return_date'].day, journeyDetails['return_date'].year)
    # Add any additional code to customize flight selection if needed

# Define the remaining functions as mentioned in the previous responses

# Define the remaining functions

def AmendTimeAndDate():
    # Code to amend time and date format
    pass

def SaveDfToCsv(journeyDetails, i):
    # Code to save DataFrame to CSV file
    filename = f"journey_{i}.csv"
    df.to_csv(filename, index=False)
    print(f"Journey details saved to {filename}")

def CheckLastFlightIndexByPrice(price_list, multiplier):
    # Code to check the index of the last flight based on price differences
    price_diff = [float(price_list[i + 1]) - float(price_list[i]) for i in range(len(price_list) - 1)]
    sorted_indexes = sorted(range(len(price_diff)), key=lambda k: price_diff[k], reverse=True)
    last_flight_index = sorted_indexes[:multiplier]
    return max(last_flight_index)

def GenerateBadIndex(dep_times_list, last_flight_index):
    # Code to generate bad indexes based on specific conditions
    return [i for i in range(len(dep_times_list)) if i not in last_flight_index]

# Call the ProcessJourney function with appropriate journey details

journeyDetails = {
    'departure_city': 'Warsaw',
    'arrival_city': 'Singapore',
    'departure_date': datetime(2023, 5, 22),
    'return_date': datetime(2023, 5, 29)
}

ProcessJourney(journeyDetails, 0)

                 

SyntaxError: expected 'except' or 'finally' block (2484557641.py, line 182)

In [ ]:
def AddFlightDetailsData(i, stops_list, bad_indexes, number_element_to_ignore):
    # Code to add flight details data
    for i in range(len(stops_list)):
        if i not in bad_indexes:
            try:
                stops = int(stops_list[i])
                if stops > number_element_to_ignore:
                    # Add your code to process and add flight details data here
                    # Example: Extract layover details, flight numbers, etc.
                    # And store the information in the respective columns of the DataFrame
                    layovers = layovers_list[i].split(' ')
                    layovers = [layover.strip() for layover in layovers if layover.strip() != '-']
                    df.loc[i, 'layovers'] = ', '.join(layovers)
            except Exception as e:
                pass

In [ ]:
# 2.0
# Define the AddFlightDetailsData function

from datetime import datetime


def AddFlightDetailsData(i, stops_list, bad_indexes, number_element_to_ignore):
    # Code to add flight details data
    for i in range(len(stops_list)):
        if i not in bad_indexes:
            try:
                stops = int(stops_list[i])
                if stops > number_element_to_ignore:
                    # Add your code to process and add flight details data here
                    # Example: Extract layover details, flight numbers, etc.
                    # And store the information in the respective columns of the DataFrame
                    pass
            except Exception as e:
                pass


# Define the remaining functions as mentioned in the previous responses

# 2.4

# Define the remaining functions

def AmendTimeAndDate():
    # Code to amend time and date format
    pass

def SaveDfToCsv(journeyDetails, i):
    # Code to save DataFrame to CSV file
    pass

def CheckLastFlightIndexByPrice(price_list, multiplier):
    # Code to check the index of the last flight based on price differences
    return 0

def GenerateBadIndex(dep_times_list, last_flight_index):
    # Code to generate bad indexes based on specific conditions
    return []

def AddFlightDetailsData(i, stops_list, bad_indexes, number_element_to_ignore):
    # Code to add flight details data
    for i in range(len(stops_list)):
        if i not in bad_indexes:
            try:
                stops = int(stops_list[i])
                if stops > number_element_to_ignore:
                    # Add your code to process and add flight details data here
                    # Example: Extract layover details, flight numbers, etc.
                    # And store the information in the respective columns of the DataFrame
                    pass
            except Exception as e:
                pass

# Define the ChooseFlight function

def ChooseFlight(journeyDetails, i):
    # Code to choose the flight
    ticket_chooser(return_ticket)
    more_details(details)
    dep_country_chooser(journeyDetails['departure_city'])
    arrival_country_chooser(journeyDetails['arrival_city'])
    dep_date_chooser(journeyDetails['departure_date'].month, journeyDetails['departure_date'].day, journeyDetails['departure_date'].year)
    return_date_chooser(journeyDetails['return_date'].month, journeyDetails['return_date'].day, journeyDetails['return_date'].year)
    # Add any additional code to customize flight selection if needed

# Call the ProcessJourney function with appropriate journey details

journeyDetails = {
    'departure_city': 'Warsaw',
    'arrival_city': 'Singapore',
    'departure_date': datetime(2023, 5, 22),
    'return_date': datetime(2023, 5, 29)
}

ProcessJourney(journeyDetails, 0)


# 3.0
# Define the AddFlightDetailsData function

def AddFlightDetailsData(i, stops_list, bad_indexes, number_element_to_ignore):
    # Code to add flight details data
    for i in range(len(stops_list)):
        if i not in bad_indexes:
            try:
                stops = int(stops_list[i])
                if stops > number_element_to_ignore:
                    # Add your code to process and add flight details data here
                    # Example: Extract layover details, flight numbers, etc.
                    # And store the information in the respective columns of the DataFrame
                    layovers = layovers_list[i].split(' ')
                    layovers = [layover.strip() for layover in layovers if layover.strip() != '-']
                    df.loc[i, 'layovers'] = ', '.join(layovers)
            except Exception as e:
                pass

# Define the ChooseFlight function

def ChooseFlight(journeyDetails, i):
    # Code to choose the flight
    ticket_chooser(return_ticket)
    more_details(details)
    dep_country_chooser(journeyDetails['departure_city'])
    arrival_country_chooser(journeyDetails['arrival_city'])
    dep_date_chooser(journeyDetails['departure_date'].month, journeyDetails['departure_date'].day, journeyDetails['departure_date'].year)
    return_date_chooser(journeyDetails['return_date'].month, journeyDetails['return_date'].day, journeyDetails['return_date'].year)
    # Add any additional code to customize flight selection if needed

# Define the remaining functions as mentioned in the previous responses

def AmendTimeAndDate():
    # Code to amend time and date format
    pass

def SaveDfToCsv(journeyDetails, i):
    # Code to save DataFrame to CSV file
    filename = f"journey_{i}.csv"
    df.to_csv(filename, index=False)
    print(f"Journey details saved to {filename}")

def CheckLastFlightIndexByPrice(price_list, multiplier):
    # Code to check the index of the last flight based on price differences
    price_diff = [float(price_list[i + 1]) - float(price_list[i]) for i in range(len(price_list) - 1)]
    sorted_indexes = sorted(range(len(price_diff)), key=lambda k: price_diff[k], reverse=True)
    last_flight_index = sorted_indexes[:multiplier]
    return max(last_flight_index)

def GenerateBadIndex(dep_times_list, last_flight_index):
    # Code to generate bad indexes based on specific conditions
    return [i for i in range(len(dep_times_list)) if i not in last_flight_index]

# Call the ProcessJourney function with appropriate journey details

journeyDetails = {
    'departure_city': 'Warsaw',
    'arrival_city': 'Singapore',
    'departure_date': datetime(2023, 5, 22),
    'return_date': datetime(2023, 5, 29)
}

ProcessJourney(journeyDetails, 0) #tu się urywa, dokończyć


NameError: name 'ProcessJourney' is not defined

In [3]:
#
print("Hello")


Hello


In [ ]:
#ZMIANA
#ZAPISUJE
#RUN
#REPEAT
#COMMIT
#PUSH
#CHECK
#CALL DAWID

#INFRASTRUKTURA SIECIOWA
#OPROGRAMOWANIE (MACHINE LEARNING, ..)
#NIE ROBIĆ: 